In [1]:
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
import json
pd.options.display.max_columns = None

In [3]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials ## para poner mis credenciales

# Importing pickle file

In [47]:
with open('../data/pickle/artist.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [48]:
tracks.head()

,url,artist_clean,track_clean
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [49]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.head()

,url,artist,track
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# spotify
Aquí está la docu: https://spotipy.readthedocs.io/en/2.22.0/#

Configuro el cliente y le metó las keys para acceder a la API de Spotify y poder sacar datos de las canciones que tengo

In [8]:
load_dotenv()

True

In [9]:
id = os.getenv("nica-id")
secret = os.getenv("nica-secret")

client_credentials_manager = SpotifyClientCredentials(client_id=id, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

See if Spotify is available

In [ ]:
import tekore as tk

app_token = tk.request_client_token(id, secret)
spotify = tk.Spotify(app_token)

album = spotify.album('3RBULTZJ97bvVzZLpxcB0j')

# API calls

Primero voy a sacar el URI de mis canciones. Para ello uso el método de spotipy.search, que permite buscar con una query el nombre del artista y la canción.

In [10]:
## esto es para obtener una uri --> me voy a sacar los uris para mi data frame
artist = 'alan walker'
track = 'alone'
q = 'artist:{} track: {}'.format(artist, track)
results = sp.search(q=q, limit=1, type='track')
print(results['tracks']['items'][0]['uri'])

spotify:track:3LlmKSHR3Rs0Y3KHQLAYDk


In [11]:
results['tracks']['items'][0]['popularity']

75

In [12]:
results['tracks']['items'][0]['album']['release_date']

'2018-12-14'

In [13]:
for index, row in tracks.iterrows():
    print(row['artist'],'---', row['track'])
    break

migos --- bad and boujee (feat. lil uzi vert)


In [ ]:
tqdm.pandas()

uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    q = 'artist:{} track: {}'.format(row['artist'].split(',')[0], row['track'])
    res = sp.search(q=q, limit=1, type='track')
    try:
        uri = res['tracks']['items'][0]['uri']
        #print(res['tracks']['items'][0]['uri'])
        uris.append(uri)
    except:
        uris.append('')
    #break

Getting URIs + popularity + release_date

In [15]:
dict_popu_search = {
    'url':[],
    'uri':[],
    'popu':[],
    'date':[]
}

In [16]:
tqdm.pandas()

#uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    q = 'artist:{} track: {}'.format(row['artist'].split(',')[0], row['track'])
    res = sp.search(q=q, limit=1, type='track')
    dict_popu_search['url'].append(row['url'])
    try:
        dict_popu_search['uri'].append(res['tracks']['items'][0]['uri'])
        dict_popu_search['popu'].append(res['tracks']['items'][0]['popularity'])
        dict_popu_search['date'].append(res['tracks']['items'][0]['album']['release_date'])
    except:
        dict_popu_search['uri'].append('')
        dict_popu_search['popu'].append('')
        dict_popu_search['date'].append('')
    #break

100%|██████████| 14659/14659 [26:12<00:00,  9.32it/s]


I can generate a dictionary from the output of the spotify api, so I will merge it later to my df.

In [18]:
df_popu_search = pd.DataFrame(dict_popu_search)
df_popu_search.head()

,url,uri,popu,date
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,76,2017-01-27
1,https://open.spotify.com/track/343YBumqHu19cGo...,spotify:track:343YBumqHu19cGoGARUTsd,75,2017-03-18
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,spotify:track:7MXVkk9YMctZqd1Srtv4MB,92,2016-11-25
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,86,2016-07-29
4,https://open.spotify.com/track/6fujklziTHa8uoM...,spotify:track:6fujklziTHa8uoM5OQSfIo,77,2016-08-12


Ahora con el método `'audio_features'` de spotipy quiero sacar las propiedades que tienen las canciones. Para ello, tengo que aportar:
- `URI`

Y el resultado de cada una de mis consultas, lo almacenaré en una lista de listas que luego appendearé a mi dataframe.

In [19]:
sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8')

[{'danceability': 0.644,
  'energy': 0.728,
  'key': 8,
  'loudness': -5.75,
  'mode': 1,
  'speechiness': 0.0351,
  'acousticness': 0.0543,
  'instrumentalness': 0.169,
  'liveness': 0.161,
  'valence': 0.43,
  'tempo': 88.993,
  'type': 'audio_features',
  'id': '3OHfY25tqY28d16oZczHc8',
  'uri': 'spotify:track:3OHfY25tqY28d16oZczHc8',
  'track_href': 'https://api.spotify.com/v1/tracks/3OHfY25tqY28d16oZczHc8',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3OHfY25tqY28d16oZczHc8',
  'duration_ms': 153947,
  'time_signature': 4}]

In [20]:
type(sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8'))

list

In [24]:
datos_uris = []
for index, row in tqdm(df_popu_search.iterrows(), total = df_popu_search.shape[0]):
    uri = sp.audio_features(row['uri'])
    datos_uris.append(uri)

100%|██████████| 14659/14659 [23:23<00:00, 10.44it/s]  


Y tengo mis datos sobre las canciones, pero tengo que limpiar un poco para poder hacer el append.

In [25]:
type(pd.DataFrame(datos_uris).iloc[0,0]) ## puedo sacar las claves para nombrar a mis columnas

dict

In [26]:
columnas = list(pd.DataFrame(datos_uris).iloc[0,0].keys())
columnas[:5]

['danceability', 'energy', 'key', 'loudness', 'mode']

In [45]:
df_uris = pd.DataFrame(datos_uris)
df_uris[columnas] = df_uris[0].apply(pd.Series)
print(df_uris.shape)
df_uris.head()

/var/folders/hh/kmst5tn53vd0_bthrjs9_2tr0000gn/T/ipykernel_32330/1468691045.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
/var/folders/hh/kmst5tn53vd0_bthrjs9_2tr0000gn/T/ipykernel_32330/1468691045.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
/var/folders/hh/kmst5tn53vd0_bthrjs9_2tr0000gn/T/ipykernel_32330/1468691045.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
/var/folders/hh/kmst5tn53vd0_bthrjs9_2tr0000gn/T/ipykernel_32330/1468691045.py:2: FutureWarning: The default dtype for em

(14659, 19)


,0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,"{'danceability': 0.926, 'energy': 0.666, 'key'...",0.926,0.666,11.0,-5.314,1.0,0.2440,0.0611,0.000000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
1,"{'danceability': 0.928, 'energy': 0.481, 'key'...",0.928,0.481,9.0,-9.350,0.0,0.2870,0.1050,0.000000,0.176,0.613,134.007,audio_features,343YBumqHu19cGoGARUTsd,spotify:track:343YBumqHu19cGoGARUTsd,https://api.spotify.com/v1/tracks/343YBumqHu19...,https://api.spotify.com/v1/audio-analysis/343Y...,210937.0,4.0
2,"{'danceability': 0.679, 'energy': 0.587, 'key'...",0.679,0.587,7.0,-7.015,1.0,0.2760,0.1410,0.000006,0.137,0.486,186.003,audio_features,7MXVkk9YMctZqd1Srtv4MB,spotify:track:7MXVkk9YMctZqd1Srtv4MB,https://api.spotify.com/v1/tracks/7MXVkk9YMctZ...,https://api.spotify.com/v1/audio-analysis/7MXV...,230453.0,4.0
3,"{'danceability': 0.748, 'energy': 0.524, 'key'...",0.748,0.524,8.0,-5.599,1.0,0.0338,0.4140,0.000000,0.111,0.661,95.010,audio_features,7BKLCZ1jbUBVqRi2FVlTVw,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,https://api.spotify.com/v1/tracks/7BKLCZ1jbUBV...,https://api.spotify.com/v1/audio-analysis/7BKL...,244960.0,4.0
4,"{'danceability': 0.794, 'energy': 0.632, 'key'...",0.794,0.632,0.0,-6.163,1.0,0.0649,0.1420,0.000000,0.128,0.355,145.926,audio_features,6fujklziTHa8uoM5OQSfIo,spotify:track:6fujklziTHa8uoM5OQSfIo,https://api.spotify.com/v1/tracks/6fujklziTHa8...,https://api.spotify.com/v1/audio-analysis/6fuj...,291893.0,4.0


Lo junto ahora con mi dataframe inicial y limpio las columnas que no necesite.

In [50]:
tracks.head()

,url,artist,track
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [51]:
tracks = tracks.merge(df_popu_search, left_on = 'url', right_on='url', indicator=True)
tracks['_merge'].value_counts() ## compruebo que el cruce se haga con éxito

both          14677
left_only         0
right_only        0
Name: _merge, dtype: int64

In [52]:
tracks.head()

,url,artist,track,uri,popu,date,_merge
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,76,2017-01-27,both
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,spotify:track:343YBumqHu19cGoGARUTsd,75,2017-03-18,both
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,spotify:track:7MXVkk9YMctZqd1Srtv4MB,92,2016-11-25,both
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,86,2016-07-29,both
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,spotify:track:6fujklziTHa8uoM5OQSfIo,77,2016-08-12,both


In [53]:
tracks.drop(['_merge'], axis = 1, inplace=True)
tracks = tracks.merge(df_uris, left_on = 'uri', right_on='uri', how = 'inner', indicator=True)
tracks['_merge'].value_counts() ## compruebo que el cruce se haga con éxito

both          19472
left_only         0
right_only        0
Name: _merge, dtype: int64

In [54]:
tracks.drop([0, 'uri', '_merge'], axis = 1, inplace=True)
tracks.head()

,url,artist,track,popu,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),76,2017-01-27,0.926,0.666,11.0,-5.314,1.0,0.244,0.0611,0.0,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
1,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),76,2017-01-27,0.926,0.666,11.0,-5.314,1.0,0.244,0.0611,0.0,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
2,https://open.spotify.com/track/1FvU97lrWOG2NRx...,migos,bad and boujee (feat. lil uzi vert),76,2017-01-27,0.926,0.666,11.0,-5.314,1.0,0.244,0.0611,0.0,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
3,https://open.spotify.com/track/1FvU97lrWOG2NRx...,migos,bad and boujee (feat. lil uzi vert),76,2017-01-27,0.926,0.666,11.0,-5.314,1.0,0.244,0.0611,0.0,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
4,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,75,2017-03-18,0.928,0.481,9.0,-9.350,0.0,0.287,0.1050,0.0,0.176,0.613,134.007,audio_features,343YBumqHu19cGoGARUTsd,https://api.spotify.com/v1/tracks/343YBumqHu19...,https://api.spotify.com/v1/audio-analysis/343Y...,210937.0,4.0


# Export

In [55]:
with open('../data/pickle/spotify.pickle', 'wb') as data_spotify:
    pickle.dump(tracks, data_spotify)